In [1]:
from ctypes import *
import numpy as np
import random

from   scipy import sparse

In [2]:
'''# speed comparison of four approaches    '''
'''    # native python libraries                        '''
'''    # specialist libraries                                   '''
'''    # ctype method calls into C                                      '''
'''    # wholesale modular call into C from compiled library                    '''
'''                                                                                 '''
'''# further optimisations maybe possible with the four approaches                      '''
'''# note that mutate request internally casts the numpy population into a list of lists    '''

'# note that mutate request internally casts the numpy population into a list of lists    '

In [3]:
class MutateRequest:
    def __init__(self, population):
        number_of_mutations = get_number_of_mutations()
        self.population     = population
        self.population     = [list(algo) for algo in population]
        self.rows_algos     = len(population)
        self.columns_genes  = len(population[0])
        self.number_of_mutations = number_of_mutations

In [4]:
class TellObject:
    def __init__(self, population, base_mutants, numpy_mutants, ctype_mutants):
        self.population     = population
        self.base_mutants   = base_mutants
        self.numpy_mutants  = numpy_mutants
        self.ctype_mutants  = ctype_mutants
        
def tell(tell_object):
    print('\noriginal population\n', np.around(np.asarray(tell_object.population), decimals=4))
    print('\nmutation by base libraries')
    for i in range(len(tell_object.base_mutants)):
          print(np.around(np.asarray(tell_object.base_mutants[i]),decimals=4))
    print('\nmutation by numpy/scipy libraries\n', np.around(np.asarray(tell_object.numpy_mutants), decimals=4))
    print('\nmutation by ctype library\n', np.around(np.asarray(tell_object.ctype_mutants), decimals=4))

In [5]:
def get_number_of_mutations():
    number_of_mutations = 3
    return number_of_mutations

In [6]:
def build_population():
    low         = -2
    high        =  2
    cohort_size = (20, 500) #  rows_algos, columns_genes
    population  = np.random.uniform(low, high, size=cohort_size)
    return population

In [7]:
def base_mutate(mutate_request):
    ''' does not use numpy; only uses naive base library - math, itertools, random etc are allowed'''
    population          = mutate_request.population
    number_of_mutations = mutate_request.number_of_mutations
    rows_algos          = mutate_request.rows_algos
    columns_genes       = mutate_request.columns_genes
    mutants        = population
    
    # arbitrary loop nesting order here - this is best for 
    for mutation in range(number_of_mutations):
        for algo in mutants:
            random_place    = random.randrange(columns_genes)
            starting_gene   = algo[random_place]
            a_float         = random.random()
            algo.pop(random_place)
            mutant_gene     = starting_gene + a_float
            algo.insert(random_place, mutant_gene)
    return mutants

In [8]:
def numpy_mutate(mutate_request):
    population          = mutate_request.population
    population          = np.asarray(population)
    
    number_of_mutations = mutate_request.number_of_mutations
    all_loci            = mutate_request.rows_algos * mutate_request.columns_genes
    
    mutation_rate       = number_of_mutations/mutate_request.columns_genes
    
    mutation_matrix = sparse.random(population.shape[0], population.shape[1], density=mutation_rate)
    pre_mutants     = np.copy(population)
    mutants         = np.add(pre_mutants, mutation_matrix.todense())
    return mutants


In [9]:
def ctype_mutate(mutate_request):
    
    '''
    # intent is to make a flat array of all genes
    # and an array of random ints equal in length to number of mutations needed and in bounds to max length flat 
    # then use the random intents to slice incremented floats into the flat array of all genes
    '''
    population          = mutate_request.population
    flat_population     = np.ravel(np.asarray(population))
    population_length   = len(flat_population)
    mutations_needed    =  mutate_request.rows_algos * mutate_request.number_of_mutations

    # declare C array     double c_flat_population  as double flat_population[population_length]
    # declare C array     c_index_stack             as int index_stack[mutations needed]
    # for loop (i=0; i<mutations_needed; ++i)
    #     index_stack[i] = random int in bounds of length of flat pop
    #     get a small float
    #     use pointer + random int of index stack[i] to retrieve, increment and replace double at [i]
    
    
    # whap the quickly incremented C array back into pytohn land
    # reshape that into population ndarray shape
    # bosh
    mutants = population
    return mutants

def test_ctype_mutate():
    population            = build_population()
    mutate_request        = MutateRequest(population)
    ctype_mutants         = ctype_mutate(mutate_request)
    
    assert False
    print('dud test for immature function')
    
#test_ctype_mutate()

In [10]:
if __name__ == '__main__':
    
    population          = build_population()
    mutate_request      = MutateRequest(population)
    base_mutants        = base_mutate(mutate_request)
    numpy_mutants       = numpy_mutate(mutate_request)
    ctype_mutants       = ctype_mutate(mutate_request)
    
    that_which_to_tell = TellObject(population, base_mutants, numpy_mutants, ctype_mutants)
    tell(that_which_to_tell)
    


original population
 [[ 1.5368 -0.7332 -0.9464 ...  1.3449  0.8218 -0.4347]
 [ 0.5301  1.9172 -0.743  ...  1.1605 -0.1283 -0.4575]
 [-1.8476  1.843  -0.8956 ...  1.4207  0.8062  1.591 ]
 ...
 [ 1.8226  0.2559 -0.3906 ...  0.9444 -0.4168 -0.3457]
 [-1.6876  0.4035 -0.8774 ...  0.8457 -1.7414  0.359 ]
 [ 1.8972  1.8241  1.8429 ...  0.0859 -0.1609 -0.1198]]

mutation by base libraries
[ 1.5368 -0.7332 -0.9464 -1.6751 -1.1341 -0.1687 -1.671  -1.6641 -0.6298
 -1.06   -0.8658  0.9182 -1.1076  1.3448 -1.918  -0.833   0.2651  1.2472
 -1.2986  1.3916 -0.3874  1.8825  0.6887 -0.7476  0.7202  0.5597  1.6826
 -0.6169  1.5737 -1.6645 -1.8752  1.1043 -1.6883 -0.3747  1.2219  1.9985
  1.5161 -0.9067 -1.9058 -0.1554 -0.5162  1.9235  1.9781  0.1787 -1.5837
 -0.341   0.3371  1.741   1.5288  0.6619  0.0031  1.8999  1.7421  0.6178
 -0.0182  0.6091 -0.4275  0.1731 -0.9632  0.6357 -0.2129  0.9549  1.52
  0.5939 -0.9734  0.1406  0.6627  1.3114  1.0012  0.1644 -1.5221 -0.425
  0.4331  1.9877 -1.7152 -0.3013 

In [11]:
def test_base_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = base_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      list),  'Error: should be a list'
    assert isinstance(mutants[0],   list),  'Error: should be a list'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')

In [12]:
def test_numpy_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = numpy_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0][0],np.matrix), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    

In [13]:
def test_build_population():
    population = build_population()
    assert isinstance(population,       np.ndarray),    'Error: should be array of arrays'
    assert isinstance(population[0],    np.ndarray),    'Error: should be an array'
    assert isinstance(population[0][0], np.float64),    'Error: should be a float'
    print('pass')

In [14]:
def tests():
    test_base_mutate()
    test_build_population()
    test_numpy_mutate()
    #test_ctype_mutate()

In [15]:
tests()

pass
pass
pass


In [16]:
'''
# Build mutate_interface
'''

'\n# Build mutate_interface\n'

In [17]:
'''# Note

 class ctypes.Array(*args)

    Abstract base class for arrays.

    The recommended way to create concrete array types is by multiplying any ctypes data type
    with a positive integer. Alternatively, you can subclass this type and define _length_ 
    and _type_ class variables. Array elements can be read and written using standard subscript 
    and slice accesses; for slice reads, the resulting object is not itself an Array.

'''

'# Note\n\n class ctypes.Array(*args)\n\n    Abstract base class for arrays.\n\n    The recommended way to create concrete array types is by multiplying any ctypes data type\n    with a positive integer. Alternatively, you can subclass this type and define _length_ \n    and _type_ class variables. Array elements can be read and written using standard subscript \n    and slice accesses; for slice reads, the resulting object is not itself an Array.\n\n'

In [18]:
cdll.LoadLibrary("libc.so.6")

<CDLL 'libc.so.6', handle 7f23bb13a500 at 0x7f239c10fd90>

In [19]:
libc = CDLL("libc.so.6")

In [20]:
c_wchar_p("Hello, World")

c_wchar_p(139790918550832)

In [21]:
printf = libc.printf

In [22]:
printf()

6

In [23]:
printf("Hello World")

1

In [24]:
s = "Hello, World"

In [25]:
c_s = c_wchar_p(s)

In [26]:
printf(b"Hello, %s\n", b"World!")

14

In [27]:
print(c_s.value)

Hello, World


In [28]:
print(c_s)

c_wchar_p(139790918994544)


In [29]:
print(s)                # first object is unchanged

Hello, World


In [30]:
printf(b"An int %d, a double %f\n", 1234, c_double(3.14))

31

In [31]:
strchr = libc.strchr

In [32]:
strchr(b"abcdef", ord("d"))  

-1676532093

In [33]:
i = c_int()

In [34]:
f = c_float()

In [35]:
s = create_string_buffer(b'\000' * 32)

In [36]:
print(i.value, f.value, repr(s.value))

0 0.0 b''


In [37]:
libc.sscanf(b"1 3.14 Hello", b"%d %f %s", byref(i), byref(f), s)

3

In [38]:
print(i.value, f.value, repr(s.value))

1 3.140000104904175 b'Hello'


In [39]:
class POINT(Structure):
    _fields_ = [("x", c_int),
                ("y", c_int)]

In [40]:
point = POINT(10, 20)

In [41]:
print(point.x, point.y)

10 20


In [42]:
point = POINT(y=5)

In [43]:
print(point.x, point.y)

0 5


In [44]:
POINT(1, 2)

In [45]:
POINT(-1, 2)

In [46]:
#POINT(1, 2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-120-a3808173cd67> in <module>
----> 1 POINT(1, 2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-120-a3808173cd67> in <module>\n----> 1 POINT(1, 2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [47]:
# POINT(1, -2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-121-bfc065603ff4> in <module>
----> 1 POINT(1, -2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-121-bfc065603ff4> in <module>\n----> 1 POINT(1, -2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [48]:
POINT(1, 2)

In [49]:
class RECT(Structure):
    _fields_ = [("upperleft", POINT),
                ("lowerright", POINT)]

In [50]:
rc = RECT(point)

In [51]:
print(rc.upperleft.x, rc.upperleft.y)

0 5


In [52]:
print(rc.lowerright.x, rc.lowerright.y)

0 0


In [53]:
TenPointsArrayType = POINT * 10

In [54]:
r = RECT(POINT(1, 2), POINT(3, 4))

In [55]:
r = RECT((1, 2), (3, 4))

In [56]:
print(POINT.x)

<Field type=c_int, ofs=0, size=4>


In [57]:
print(POINT.y)

<Field type=c_int, ofs=4, size=4>


In [58]:
class Int(Structure):
    _fields_ = [("first_16", c_int, 16),
                ("second_16", c_int, 16)]

In [59]:
print(Int.first_16)

<Field type=c_int, ofs=0:0, bits=16>


In [60]:
print(Int.second_16)

<Field type=c_int, ofs=0:16, bits=16>


In [61]:
class POINT(Structure):
    _fields_ = ("x", c_int), ("y", c_int)

In [62]:
class MyStruct(Structure):
    _fields_ = [("a", c_int),
                ("b", c_float),
                ("point_array", POINT * 4)]

In [63]:
print(len(MyStruct().point_array))

4


In [64]:
arr = TenPointsArrayType()

In [65]:
for pt in arr:
    print(pt.x, pt.y)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [66]:
TenIntegers = c_int * 10

In [67]:
ii = TenIntegers(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [68]:
print(ii)

In [69]:
for i in ii: print(i, end=" ")

1 2 3 4 5 6 7 8 9 10 

In [70]:
i = c_int(42)

In [71]:
pi = pointer(i)

In [72]:
pi.contents

c_int(42)

In [73]:
pi.contents is i

False

In [74]:
pi.contents is pi.contents

False

In [75]:
i = c_int(99)

In [76]:
pi.contents = i

In [77]:
pi.contents

c_int(99)

In [78]:
pi[0]

99

In [79]:
print(i)

c_int(99)


In [80]:
pi[0] = 22

In [81]:
print(i)

c_int(22)


In [82]:
PI = POINTER(c_int)

In [83]:
PI

importlib._bootstrap.LP_c_int

In [84]:
# PI(42)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-52-622c40ff30d9> in <module>
----> 1 PI(42)

TypeError: expected c_int instead of int


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-52-622c40ff30d9> in <module>\n----> 1 PI(42)\n\nTypeError: expected c_int instead of int\n\n\n'

In [85]:
PI(c_int(42))

In [86]:
null_ptr = POINTER(c_int)()

In [87]:
print(bool(null_ptr))

False


In [88]:
class Bar(Structure):
    _fields_ = [("count", c_int), ("values", POINTER(c_int))]

In [89]:
bar = Bar()

In [90]:
bar.values = (c_int * 3)(1, 2, 3)

In [91]:
bar.count = 3

In [92]:
for i in range(bar.count):
    print(bar.values[i])

1
2
3


In [93]:
IntArray5 = c_int * 5

In [94]:
ia = IntArray5(5, 1, 7, 33, 99)

In [95]:
qsort = libc.qsort

In [96]:
qsort.restype = None

In [97]:
CMPFUNC = CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))



In [98]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return 0

In [99]:
cmp_func = CMPFUNC(py_cmp_func)

In [100]:
qsort(ia, len(ia), sizeof(c_int), cmp_func) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 5 7
py_cmp_func 1 7


In [101]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [102]:
qsort(ia, len(ia), sizeof(c_int), CMPFUNC(py_cmp_func)) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [103]:
for i in ia: print(i, end=" ")




1 5 7 33 99 

In [104]:
@CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [105]:
qsort(ia, len(ia), sizeof(c_int), py_cmp_func)

py_cmp_func 1 5
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [106]:
short_array = (c_short * 4)()

In [107]:
print(sizeof(short_array))

8


In [108]:
#resize(short_array, 4)

'''
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-207-5d3303a611a5> in <module>
----> 1 resize(short_array, 4)
      2 
      3 

ValueError: minimum size is 8

'''

'\n---------------------------------------------------------------------------\nValueError                                Traceback (most recent call last)\n<ipython-input-207-5d3303a611a5> in <module>\n----> 1 resize(short_array, 4)\n      2 \n      3 \n\nValueError: minimum size is 8\n\n'

In [109]:
resize(short_array, 32)

In [110]:
sizeof(short_array)

32

In [111]:
sizeof(type(short_array))

8